In [1]:
import simi

import ast
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import MultiLabelBinarizer
from sentence_transformers import evaluation, losses, models, InputExample, SentenceTransformer, util as stu
from sklearn.metrics import classification_report, f1_score, jaccard_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
import torch
from tqdm.auto import tqdm, trange
tqdm.pandas()

pd.set_option('display.max_columns', 30)

RANDOM_STATE = 1

random.seed(RANDOM_STATE)

2024-01-02 12:37:45.398888: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
def _msc_lvl(level):
    if   level==1: return 2
    elif level==2: return 3
    else:          return 5

def _lvl_overlap(r, n_results, prediction_col, level=1):
    e = _msc_lvl(level)
    actual = set([r["MSC"][0:e]]) | set([m[0:e] for m in r["MSC2"]])
    prediction = set([t[0] for t in Counter([m[0:e] for m in r[prediction_col][:n_results]]).most_common(len(actual))])
    return actual, prediction
                      
    
def msc_overlap(df, n_results, prediction_col, level=1):
    ap = df.apply(lambda r: _lvl_overlap(r, n_results, prediction_col, level=level), axis=1)
    return pd.Series(ap.map(lambda c: len(c[0] & c[1])/len(c[0])))

def msc_overlap_jaccard(df, n_results, prediction_col, level=1):
    ap = df.apply(lambda r: _lvl_overlap(r, n_results, prediction_col, level=level), axis=1)
    actual = ap.map(lambda c: list(c[0]))
    predictions = ap.map(lambda c: list(c[1]))
    trans = MultiLabelBinarizer().fit_transform(pd.concat([actual, predictions]))
    #print(trans[0:int(len(trans)/2)], trans[int(len(trans)/2):len(trans)])
    return jaccard_score(trans[0:int(len(trans)/2)], trans[int(len(trans)/2):len(trans)], average="weighted")

def model_msc_overlap(eval_models, df, n_results, level=1, prediction_col="MSC"):
    res = pd.DataFrame({"MSC overlap ({})".format(n): [msc_overlap(df, n_results, "{} ({})".format(prediction_col, n), level=level).sum()/len(df)] for n in eval_models.keys()})
    return res

def model_msc_overlap_jaccard(eval_models, df, model_params, level=1, prediction_col="MSC"):
    res = pd.DataFrame({"jaccard ({})".format(n): [msc_overlap_jaccard(df, model_params[n][0], "{} ({})".format(prediction_col, n), level=level)] for n in eval_models.keys()})
    return res

def model_msc_overlap_separate_params(eval_models, df, model_params, level=1, prediction_col="MSC"):
    res = pd.DataFrame({"MSC overlap ({})".format(n): [msc_overlap(df, model_params[n][0], "{} ({})".format(prediction_col, n), level=level).sum()/len(df)] for n in eval_models.keys()})
    return res

def model_msc_overlap_param(eval_models, df, n_range, level=1, prediction_col="MSC"):
    res = pd.concat([model_msc_overlap(eval_models, df, n, level=level, prediction_col=prediction_col) for n in tqdm(n_range, leave=False)], ignore_index=True)
    res.index = n_range
    return res

def best_model_msc_overlap_param(eval_models, df, n_range, level=1, prediction_col="MSC"):
    params = model_msc_overlap_param(eval_models, df, n_range, level=level, prediction_col=prediction_col)
    best = {}
    for n in eval_models.keys():
        cn = "MSC overlap ({})".format(n)
        print(n, list(params[cn].sort_values(ascending=False).items())[:5])
        best[n] = (params[cn].idxmax(), params[cn].max())
    return best


def model_print_msc_overlap(eval_models, df, model_params, level=1):
    for n in eval_models.keys():
        s = msc_overlap(df, model_params[n][0], "MSC ({})".format(n), level=level)
        print(n)
        print("\tmind 1 match", len(s.loc[lambda x: x>0])/len(s))
        print("\t0% overlap", len(s.loc[lambda x: x==0])/len(s))
        print("\t(0%-50%)", len(s.loc[lambda x: (x>0) & (x<0.5)])/len(s))
        print("\t(0%-100%)", len(s.loc[lambda x: (x>0.5) & (x<1)])/len(s))
        #print("\t(0%-33%)", len(s.loc[lambda x: (x>0) & (x<0.33)])/len(s))
        #print("\t[33%-50%)", len(s.loc[lambda x: (x>=0.33) & (x<0.5)])/len(s))
        #print("\t[50%-66%)", len(s.loc[lambda x: (x>=0.5) & (x<0.66)])/len(s))
        #print("\t[66%-100%)", len(s.loc[lambda x: (x>=0.66) & (x<1)])/len(s))
        print("\t100%", len(s.loc[lambda x: x==1])/len(s))

In [3]:
# Prerequisites for the models
bert_base = models.Transformer('bert-base-uncased')
bert_mp = SentenceTransformer(modules=[bert_base, models.Pooling(bert_base.get_word_embedding_dimension())], device="cuda:1")

mathbert_base = models.Transformer('witiko/mathberta')
mathbert_mp = SentenceTransformer(modules=[mathbert_base, models.Pooling(mathbert_base.get_word_embedding_dimension())], device="cuda:1")
mathbert_cls = SentenceTransformer(modules=[mathbert_base, models.Pooling(mathbert_base.get_word_embedding_dimension(), pooling_mode="cls")], device="cuda:1")

bert_mlm_base = models.Transformer("./bert+re-train_mlm_abstracts_arxiv")
bert_mlm_mp = SentenceTransformer(modules=[bert_mlm_base, models.Pooling(bert_mlm_base.get_word_embedding_dimension())], device="cuda:1")

sbert = SentenceTransformer('all-mpnet-base-v2', device="cuda:1")
# alternative: sbert = SentenceTransformer('all-distilroberta-v1')

specter2_base = models.Transformer('allenai/specter2_base')

# Definition of models that are evaluated
# combine models into dict
eval_models = {
    "Bert+MP": bert_mp,
    "Bert+MP+class-arx": SentenceTransformer('./bert+mean-pooling+retrain_class_arxiv', device="cuda:1"),
    "Bert+MP+class-zbm": SentenceTransformer('./bert+mean-pooling+retrain_class_zbmath', device="cuda:1"),
    
    "Mathbert+CLS": mathbert_cls,
    "Mathbert+MP+class-arx": SentenceTransformer('./mathbert+mean-pooling+retrain_class_arxiv', device="cuda:1"),
    "Mathbert+MP+class-zbm": SentenceTransformer('./mathbert+mean-pooling+retrain_class_zbmath', device="cuda:1"),
    
    "Bert+TSDAE+MP": SentenceTransformer('./bert+mean-pooling+re-train_tsdae_abstracts_arxiv', device="cuda:1"),
    "Bert+TSDAE+MP+class-arx": SentenceTransformer('./bert+mean-pooling+re-train_tsdae_abstracts_arxiv+retrain_class_arxiv', device="cuda:1"),
    "Bert+TSDAE+MP+class-zbm": SentenceTransformer('./bert+mean-pooling+re-train_tsdae_abstracts_arxiv+retrain_class_zbmath', device="cuda:1"),
    
    "Bert+MLM+MP": bert_mlm_mp,
    "Bert+MLM+MP+class-arx": SentenceTransformer('./bert+re-train_mlm_abstracts_arxiv+mean-pooling+retrain_class_arxiv', device="cuda:1"),
    "Bert+MLM+MP+class-zbm": SentenceTransformer('./bert+re-train_mlm_abstracts_arxiv+mean-pooling+retrain_class_zbmath', device="cuda:1"),
    "Bert+MLM+MP+class-zbm+anch-arx": SentenceTransformer('./bert+re-train_mlm_abstracts_arxiv+mean-pooling+retrain_class_zbmath_anchor_arxiv', device="cuda:1"),
    "Bert+MLM+MP+class-arx+anch-arx+class-zbm": SentenceTransformer('./bert+re-train_mlm_abstracts_arxiv+mean-pooling+retrain_class_arxiv_anchor_arxiv_class_zbmath', device="cuda:1"),
    
    "SBert": sbert,
    "SBert+ret_class-arx": SentenceTransformer('./sbert+retrain_class_arxiv', device="cuda:1"),
    "SBert+ret_class-zbm": SentenceTransformer('./sbert+retrain_class_zbmath', device="cuda:1"),
    
    "Specter 2+MP": SentenceTransformer(modules=[specter2_base, models.Pooling(specter2_base.get_word_embedding_dimension())], device="cuda:1"),
    "Specter 2.0+MP+class-arx": SentenceTransformer('./specter2+mp+retrain_class_arxiv', device="cuda:1"),
    "Specter 2.0+MP+class-zbm": SentenceTransformer('./specter2+mp+retrain_class_zbmath', device="cuda:1"),
    "Specter 2+MP+class-zbm+anch-arx": SentenceTransformer('./specter2+mp+retrain_class_zbmath_anchor_arxiv', device="cuda:1"),
    "Specter 2+MP+anch-arx+class-arx+class-zbm": SentenceTransformer('./specter2+mp+retrain_anchor_arxiv_class_arxiv_class_zbmath', device="cuda:1"),
    
    "SGPT": SentenceTransformer('Muennighoff/SGPT-125M-weightedmean-nli-bitfit', device="cuda:1"),
    "SGPT+class-arx": SentenceTransformer('./gpt+retrain_class_arxiv', device="cuda:1"),
    "SGPT+class-zbm": SentenceTransformer('./gpt+retrain_class_zbmath', device="cuda:1"),
}

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at witiko/mathberta were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_he

In [4]:
%%time
X_test = pd.read_pickle("zbmath-200-recommendations-2000-2019-multi-test.pkl")
X_eval = pd.read_pickle("zbmath-200-recommendations-2000-2019-multi-eval.pkl")

X_test["MSC2"] = X_test["MSC2"].map(lambda c: ast.literal_eval(c))
X_eval["MSC2"] = X_eval["MSC2"].map(lambda c: ast.literal_eval(c))

CPU times: user 1min 2s, sys: 14.2 s, total: 1min 17s
Wall time: 1min 16s


In [5]:
# Average amount of MSC_1 per article
print("Number of distinct primary MSC_1:", len(X_test["MSC"].map(lambda c: c[0:2]).unique()))
X_test.apply(lambda r: len(set([a[0:2] for a in r["MSC2"]] + [r["MSC"][0:2]])), axis=1).mean(), X_eval.apply(lambda r: len(set([a[0:2] for a in r["MSC2"]] + [r["MSC"][0:2]])), axis=1).mean()

Number of distinct primary MSC_1: 61


(1.7831184399029325, 1.7870654620939683)

In [6]:
# Average amount of MSC_2 per article
print("Number of distinct primary MSC_2:", len(X_test["MSC"].map(lambda c: c[0:3]).unique()))
X_test.apply(lambda r: len(set([a[0:3] for a in r["MSC2"]] + [r["MSC"][0:3]])), axis=1).mean(), X_eval.apply(lambda r: len(set([a[0:3] for a in r["MSC2"]] + [r["MSC"][0:3]])), axis=1).mean()

Number of distinct primary MSC_2: 484


(2.494761668079798, 2.500238770478223)

In [7]:
# Average amount of MSC_3 per article
print("Number of distinct primary MSC_3:", len(X_test["MSC"].unique()))
X_test["MSC2"].map(lambda c: len(c)+1).mean(), X_eval["MSC2"].map(lambda c: len(c)+1).mean()

Number of distinct primary MSC_3: 3822


(3.186884191445195, 3.189164693155571)

## Parameter search

In [10]:
%%time
best_1 = best_model_msc_overlap_param(eval_models, X_test, range(15, 61), level=1, prediction_col="MSC")

  0%|          | 0/46 [00:00<?, ?it/s]

Bert+MP [(20, 0.6059058528081713), (19, 0.6058171666305454), (21, 0.605329392653604), (18, 0.6052705933929804), (22, 0.6050596910726139)]
Bert+MP+class-arx [(24, 0.5501460429149522), (22, 0.5500656947540263), (23, 0.5499729478539817), (21, 0.5499573546798936), (25, 0.549756213562751)]
Bert+MP+class-zbm [(32, 0.6453612511480242), (31, 0.6451409975640305), (26, 0.6449258720923486), (25, 0.6448858217672357), (33, 0.6448424532518032)]
Mathbert+CLS [(31, 0.6534500748487825), (30, 0.6532451359893396), (32, 0.6532374554229363), (33, 0.6531961984831616), (27, 0.6529780216686336)]
Mathbert+MP+class-arx [(38, 0.5382054886425849), (27, 0.5380526082445505), (24, 0.5380302858733114), (29, 0.5380019304287643), (28, 0.537976336275462)]
Mathbert+MP+class-zbm [(37, 0.6309243851718196), (32, 0.6308836039166711), (33, 0.6307831532460951), (31, 0.6307638938197662), (34, 0.6306951167840562)]
Bert+TSDAE+MP [(26, 0.6907061743245703), (27, 0.6906274195137745), (28, 0.6904783097865576), (30, 0.6904728568164523

In [11]:
%%time
best_2 = best_model_msc_overlap_param(eval_models, X_test, range(15, 61), level=2, prediction_col="MSC")

  0%|          | 0/46 [00:00<?, ?it/s]

Bert+MP [(32, 0.4280700158137809), (34, 0.42797493338308473), (33, 0.4279489331575407), (31, 0.42791267635579533), (27, 0.4278819705061555)]
Bert+MP+class-arx [(30, 0.3568939084632152), (31, 0.3568279275249424), (33, 0.3566768454468363), (37, 0.3565713208069141), (29, 0.3565416195229369)]
Bert+MP+class-zbm [(32, 0.4819287819419388), (42, 0.48188125602022047), (40, 0.4818625805644487), (36, 0.48182429762042656), (39, 0.4818047094689494)]
Mathbert+CLS [(38, 0.47363057754909343), (39, 0.47357810528073374), (37, 0.47344849862319094), (41, 0.4734263618849766), (40, 0.4734061858955874)]
Mathbert+MP+class-arx [(41, 0.33979700740438096), (40, 0.33962738522791736), (42, 0.33961322684241735), (39, 0.3396019302993838), (34, 0.33955452813301534)]
Mathbert+MP+class-zbm [(43, 0.46783170126151735), (44, 0.46771261767607125), (45, 0.4676843241091991), (49, 0.46759349893285507), (57, 0.4675890824138053)]
Bert+TSDAE+MP [(36, 0.5164612847922792), (39, 0.516346822695681), (35, 0.5163296593755909), (37, 0.

In [12]:
%%time
best_3 = best_model_msc_overlap_param(eval_models, X_test, range(15, 61), level=3, prediction_col="MSC")

  0%|          | 0/46 [00:00<?, ?it/s]

Bert+MP [(41, 0.24984128287128024), (40, 0.24983818452520026), (39, 0.2498027096322567), (38, 0.2498004272980152), (35, 0.24978201517393986)]
Bert+MP+class-arx [(40, 0.18033820662537442), (33, 0.1802733862450846), (41, 0.18027007578947468), (30, 0.18025424670675083), (34, 0.18024476008570967)]
Bert+MP+class-zbm [(38, 0.30337686263336056), (39, 0.30335367045889516), (37, 0.3033399564676061), (40, 0.303334658543266), (36, 0.30331241668331976)]
Mathbert+CLS [(54, 0.28350860344325834), (52, 0.28349386530613324), (53, 0.28347719278851047), (40, 0.28343993177268034), (39, 0.2834183364641219)]
Mathbert+MP+class-arx [(54, 0.1582584765726595), (52, 0.15818365521315558), (53, 0.15816995317550828), (51, 0.15814079754011923), (45, 0.15814032572284772)]
Mathbert+MP+class-zbm [(54, 0.28595475277126003), (51, 0.28593284385539447), (55, 0.2859265678418969), (56, 0.285890721331326), (59, 0.2858842245270336)]
Bert+TSDAE+MP [(44, 0.3294253696937672), (49, 0.3294054489497871), (48, 0.32936450526574196), (

## X_test

### MSC1

In [8]:
best_1 = {
    "Bert+MP": (20, 0.6059058528081713),
    "Bert+MP+class-arx": (24, 0.5501460429149522),
    "Bert+MP+class-zbm": (32, 0.6453612511480242),
    "Mathbert+CLS": (31, 0.6534500748487825),
    "Mathbert+MP+class-arx": (38, 0.5382054886425849),
    "Mathbert+MP+class-zbm": (37, 0.6309243851718196),
    "Bert+TSDAE+MP": (26, 0.6907061743245703),
    "Bert+TSDAE+MP+class-arx": (28, 0.5249107491885904),
    "Bert+TSDAE+MP+class-zbm": (33, 0.6491383959171872),
    "Bert+MLM+MP": (38, 0.6983984472106859),
    "Bert+MLM+MP+class-arx": (30, 0.5998066980108029),
    "Bert+MLM+MP+class-zbm": (42, 0.6767309235382212),
    "Bert+MLM+MP+class-zbm+anch-arx": (43, 0.6688332477101007),
    "Bert+MLM+MP+class-arx+anch-arx+class-zbm": (40, 0.6786780631421452),
    "SBert": (40, 0.7326007596722153),
    "SBert+ret_class-arx": (23, 0.5904460483137792),
    "SBert+ret_class-zbm": (27, 0.6664441932210842),
    "Specter 2+MP": (29, 0.7155685119672196),
    "Specter 2.0+MP+class-arx": (43, 0.5354430603955499),
    "Specter 2.0+MP+class-zbm": (36, 0.6541173056851506),
    "Specter 2+MP+class-zbm+anch-arx": (35, 0.6767999674214041),
    "Specter 2+MP+anch-arx+class-arx+class-zbm": (43, 0.6786091623510857),
    "SGPT": (22, 0.6362086690003833),
    "SGPT+class-arx": (20, 0.5746606202896587),
    "SGPT+class-zbm": (31, 0.6570214879507924)
}

In [6]:
model_msc_overlap_jaccard(eval_models, X_test, best_1, level=1, prediction_col="MSC")

,jaccard (Bert+MP),jaccard (Bert+MP+class-arx),jaccard (Bert+MP+class-zbm),jaccard (Mathbert+CLS),jaccard (Mathbert+MP+class-arx),jaccard (Mathbert+MP+class-zbm),jaccard (Bert+TSDAE+MP),jaccard (Bert+TSDAE+MP+class-arx),jaccard (Bert+TSDAE+MP+class-zbm),jaccard (Bert+MLM+MP),jaccard (Bert+MLM+MP+class-arx),jaccard (Bert+MLM+MP+class-zbm),jaccard (Bert+MLM+MP+class-zbm+anch-arx),jaccard (Bert+MLM+MP+class-arx+anch-arx+class-zbm),jaccard (SBert),jaccard (SBert+ret_class-arx),jaccard (SBert+ret_class-zbm),jaccard (Specter 2+MP),jaccard (Specter 2.0+MP+class-arx),jaccard (Specter 2.0+MP+class-zbm),jaccard (Specter 2+MP+class-zbm+anch-arx),jaccard (Specter 2+MP+anch-arx+class-arx+class-zbm),jaccard (SGPT),jaccard (SGPT+class-arx),jaccard (SGPT+class-zbm)
0,0.394078,0.348064,0.438852,0.444179,0.340251,0.425302,0.479565,0.327819,0.443419,0.490703,0.395581,0.472588,0.463878,0.474703,0.53225,0.385234,0.461765,0.510867,0.336176,0.449328,0.473023,0.474222,0.425826,0.369797,0.449934


In [7]:
model_msc_overlap_separate_params(eval_models, X_test, best_1, level=1, prediction_col="MSC")

,MSC overlap (Bert+MP),MSC overlap (Bert+MP+class-arx),MSC overlap (Bert+MP+class-zbm),MSC overlap (Mathbert+CLS),MSC overlap (Mathbert+MP+class-arx),MSC overlap (Mathbert+MP+class-zbm),MSC overlap (Bert+TSDAE+MP),MSC overlap (Bert+TSDAE+MP+class-arx),MSC overlap (Bert+TSDAE+MP+class-zbm),MSC overlap (Bert+MLM+MP),MSC overlap (Bert+MLM+MP+class-arx),MSC overlap (Bert+MLM+MP+class-zbm),MSC overlap (Bert+MLM+MP+class-zbm+anch-arx),MSC overlap (Bert+MLM+MP+class-arx+anch-arx+class-zbm),MSC overlap (SBert),MSC overlap (SBert+ret_class-arx),MSC overlap (SBert+ret_class-zbm),MSC overlap (Specter 2+MP),MSC overlap (Specter 2.0+MP+class-arx),MSC overlap (Specter 2.0+MP+class-zbm),MSC overlap (Specter 2+MP+class-zbm+anch-arx),MSC overlap (Specter 2+MP+anch-arx+class-arx+class-zbm),MSC overlap (SGPT),MSC overlap (SGPT+class-arx),MSC overlap (SGPT+class-zbm)
0,0.605906,0.550146,0.645361,0.65345,0.538205,0.630924,0.690706,0.524911,0.649138,0.698398,0.599807,0.676731,0.668833,0.678678,0.732601,0.590446,0.666444,0.715569,0.535443,0.654117,0.6768,0.678609,0.636209,0.574661,0.657021


In [39]:
model_print_msc_overlap(eval_models, X_test, best_1, level=1)

Bert+MP
	mind 1 match 0.7939654416279274
	0% overlap 0.20603455837207263
	(0%-50%) 0.06416591137229678
	(0%-100%) 0.05825999668645051
	100% 0.4204699392840784
Bert+MP+class-arx
	mind 1 match 0.7402177196932043
	0% overlap 0.2597822803067957
	(0%-50%) 0.06940911615940122
	(0%-100%) 0.05257823387811985
	100% 0.36673196308316036
Bert+MP+class-zbm
	mind 1 match 0.827597968989075
	0% overlap 0.17240203101092497
	(0%-50%) 0.05688584821994172
	(0%-100%) 0.06655361615452836
	100% 0.4600766014677075
Mathbert+CLS
	mind 1 match 0.8379869212252337
	0% overlap 0.16201307877476634
	(0%-50%) 0.057285423305947825
	(0%-100%) 0.06738200352795563
	100% 0.4656219240027678
Mathbert+MP+class-arx
	mind 1 match 0.7272948766677387
	0% overlap 0.2727051233322613
	(0%-50%) 0.06828835677182313
	(0%-100%) 0.05348458712198735
	100% 0.3550663197185432
Mathbert+MP+class-zbm
	mind 1 match 0.8122289467785477
	0% overlap 0.18777105322145232
	(0%-50%) 0.05679813661569648
	(0%-100%) 0.06475065540059839
	100% 0.44725121578

### MSC2

In [9]:
best_2 = {
    "Bert+MP": (32, 0.4280700158137809),
    "Bert+MP+class-arx": (30, 0.3568939084632152),
    "Bert+MP+class-zbm": (32, 0.4819287819419388),
    "Mathbert+CLS": (38, 0.47363057754909343),
    "Mathbert+MP+class-arx": (41, 0.33979700740438096),
    "Mathbert+MP+class-zbm": (43, 0.46783170126151735),
    "Bert+TSDAE+MP": (36, 0.5164612847922792),
    "Bert+TSDAE+MP+class-arx": (38, 0.32753568662114696),
    "Bert+TSDAE+MP+class-zbm": (45, 0.48624273368813314),
    "Bert+MLM+MP": (36, 0.5182515644901985),
    "Bert+MLM+MP+class-arx": (45, 0.3963662904939985),
    "Bert+MLM+MP+class-zbm": (44, 0.5091366565703612), 
    "Bert+MLM+MP+class-zbm+anch-arx": (48, 0.5010198617265249),
    "Bert+MLM+MP+class-arx+anch-arx+class-zbm": (60, 0.5074571264167206),
    "SBert": (44, 0.5634984071977903),
    "SBert+ret_class-arx": (26, 0.39795603551926095),
    "SBert+ret_class-zbm": (36, 0.5019368770238577),
    "Specter 2+MP": (33, 0.5417367981689825),
    "Specter 2.0+MP+class-arx": (46, 0.331874559627973),
    "Specter 2.0+MP+class-zbm": (43, 0.4895919238842374),
    "Specter 2+MP+class-zbm+anch-arx": (55, 0.5088654754422379),
    "Specter 2+MP+anch-arx+class-arx+class-zbm": (51, 0.5085362819925578),
    "SGPT": (31, 0.46633524756788797),
    "SGPT+class-arx": (27, 0.3895566784903979),
    "SGPT+class-zbm": (39, 0.4908629663490348)
}

In [10]:
model_msc_overlap_jaccard(eval_models, X_test, best_2, level=2, prediction_col="MSC")

,jaccard (Bert+MP),jaccard (Bert+MP+class-arx),jaccard (Bert+MP+class-zbm),jaccard (Mathbert+CLS),jaccard (Mathbert+MP+class-arx),jaccard (Mathbert+MP+class-zbm),jaccard (Bert+TSDAE+MP),jaccard (Bert+TSDAE+MP+class-arx),jaccard (Bert+TSDAE+MP+class-zbm),jaccard (Bert+MLM+MP),jaccard (Bert+MLM+MP+class-arx),jaccard (Bert+MLM+MP+class-zbm),jaccard (Bert+MLM+MP+class-zbm+anch-arx),jaccard (Bert+MLM+MP+class-arx+anch-arx+class-zbm),jaccard (SBert),jaccard (SBert+ret_class-arx),jaccard (SBert+ret_class-zbm),jaccard (Specter 2+MP),jaccard (Specter 2.0+MP+class-arx),jaccard (Specter 2.0+MP+class-zbm),jaccard (Specter 2+MP+class-zbm+anch-arx),jaccard (Specter 2+MP+anch-arx+class-arx+class-zbm),jaccard (SGPT),jaccard (SGPT+class-arx),jaccard (SGPT+class-zbm)
0,0.238233,0.190364,0.286937,0.275833,0.1784,0.275619,0.308188,0.170165,0.290384,0.312542,0.219263,0.310409,0.303865,0.308664,0.355571,0.220704,0.304513,0.333933,0.172345,0.294251,0.310337,0.30995,0.270463,0.213306,0.293281


In [11]:
model_msc_overlap_separate_params(eval_models, X_test, best_2, level=2, prediction_col="MSC")

,MSC overlap (Bert+MP),MSC overlap (Bert+MP+class-arx),MSC overlap (Bert+MP+class-zbm),MSC overlap (Mathbert+CLS),MSC overlap (Mathbert+MP+class-arx),MSC overlap (Mathbert+MP+class-zbm),MSC overlap (Bert+TSDAE+MP),MSC overlap (Bert+TSDAE+MP+class-arx),MSC overlap (Bert+TSDAE+MP+class-zbm),MSC overlap (Bert+MLM+MP),MSC overlap (Bert+MLM+MP+class-arx),MSC overlap (Bert+MLM+MP+class-zbm),MSC overlap (Bert+MLM+MP+class-zbm+anch-arx),MSC overlap (Bert+MLM+MP+class-arx+anch-arx+class-zbm),MSC overlap (SBert),MSC overlap (SBert+ret_class-arx),MSC overlap (SBert+ret_class-zbm),MSC overlap (Specter 2+MP),MSC overlap (Specter 2.0+MP+class-arx),MSC overlap (Specter 2.0+MP+class-zbm),MSC overlap (Specter 2+MP+class-zbm+anch-arx),MSC overlap (Specter 2+MP+anch-arx+class-arx+class-zbm),MSC overlap (SGPT),MSC overlap (SGPT+class-arx),MSC overlap (SGPT+class-zbm)
0,0.42807,0.356894,0.481929,0.473631,0.339797,0.467832,0.516461,0.327536,0.486243,0.518252,0.396366,0.509137,0.50102,0.507457,0.563498,0.397956,0.501937,0.541737,0.331875,0.489592,0.508865,0.508536,0.466335,0.389557,0.490863


In [40]:
model_print_msc_overlap(eval_models, X_test, best_2, level=2)

Bert+MP
	mind 1 match 0.7148398288649144
	0% overlap 0.2851601711350856
	(0%-50%) 0.18201132454268143
	(0%-100%) 0.09761326979114893
	100% 0.17694354296406747
Bert+MP+class-arx
	mind 1 match 0.6222163747819393
	0% overlap 0.3777836252180608
	(0%-50%) 0.18222573068639203
	(0%-100%) 0.071865041078268
	100% 0.13732713504663333
Bert+MP+class-zbm
	mind 1 match 0.7797951446754183
	0% overlap 0.22020485532458167
	(0%-50%) 0.17900963853073318
	(0%-100%) 0.11786490463799472
	100% 0.21007903790115875
Mathbert+CLS
	mind 1 match 0.769864241928096
	0% overlap 0.23013575807190403
	(0%-50%) 0.17860031771092205
	(0%-100%) 0.11474626982038613
	100% 0.20451422389848845
Mathbert+MP+class-arx
	mind 1 match 0.597442719449561
	0% overlap 0.40255728055043905
	(0%-50%) 0.17918506173922366
	(0%-100%) 0.06912648987905544
	100% 0.12752292683877633
Mathbert+MP+class-zbm
	mind 1 match 0.7605083374752702
	0% overlap 0.2394916625247298
	(0%-50%) 0.17549142862711847
	(0%-100%) 0.11407381418783927
	100% 0.201171437203

### MSC3

In [10]:
best_3 = {
    "Bert+MP": (41, 0.24984128287128024),
    "Bert+MP+class-arx": (40, 0.18033820662537442),
    "Bert+MP+class-zbm": (38, 0.30337686263336056),
    "Mathbert+CLS": (54, 0.28350860344325834),
    "Mathbert+MP+class-arx": (54, 0.1582584765726595),
    "Mathbert+MP+class-zbm": (54, 0.28595475277126003),
    "Bert+TSDAE+MP": (44, 0.3294253696937672),
    "Bert+TSDAE+MP+class-arx": (41, 0.15404923375203758),
    "Bert+TSDAE+MP+class-zbm": (41, 0.30518524839344646),
    "Bert+MLM+MP": (48, 0.32041576640734615),
    "Bert+MLM+MP+class-arx": (42, 0.20071363503591666),
    "Bert+MLM+MP+class-zbm": (53, 0.31901199668135316),
    "Bert+MLM+MP+class-zbm+anch-arx": (52, 0.31235275951675096),
    "Bert+MLM+MP+class-arx+anch-arx+class-zbm": (53, 0.317714104281803),
    "SBert": (49, 0.3759015800086758),
    "SBert+ret_class-arx": (27, 0.2146777074363836),
    "SBert+ret_class-zbm": (39, 0.3202606619287904),
    "Specter 2+MP": (52, 0.35056778164838864),
    "Specter 2.0+MP+class-arx": (43, 0.1521497033667019),
    "Specter 2.0+MP+class-zbm": (51, 0.3032266622039449),
    "Specter 2+MP+class-zbm+anch-arx": (49, 0.32281059282857366),
    "Specter 2+MP+anch-arx+class-arx+class-zbm": (53, 0.3196890253120701),
    "SGPT": (38, 0.2897234270059753),
    "SGPT+class-arx": (33, 0.21216490184343884),
    "SGPT+class-zbm": (40, 0.3100256618347137),
}

In [14]:
model_msc_overlap_jaccard(eval_models, X_test, best_3, level=3, prediction_col="MSC")

,jaccard (Bert+MP),jaccard (Bert+MP+class-arx),jaccard (Bert+MP+class-zbm),jaccard (Mathbert+CLS),jaccard (Mathbert+MP+class-arx),jaccard (Mathbert+MP+class-zbm),jaccard (Bert+TSDAE+MP),jaccard (Bert+TSDAE+MP+class-arx),jaccard (Bert+TSDAE+MP+class-zbm),jaccard (Bert+MLM+MP),jaccard (Bert+MLM+MP+class-arx),jaccard (Bert+MLM+MP+class-zbm),jaccard (Bert+MLM+MP+class-zbm+anch-arx),jaccard (Bert+MLM+MP+class-arx+anch-arx+class-zbm),jaccard (SBert),jaccard (SBert+ret_class-arx),jaccard (SBert+ret_class-zbm),jaccard (Specter 2+MP),jaccard (Specter 2.0+MP+class-arx),jaccard (Specter 2.0+MP+class-zbm),jaccard (Specter 2+MP+class-zbm+anch-arx),jaccard (Specter 2+MP+anch-arx+class-arx+class-zbm),jaccard (SGPT),jaccard (SGPT+class-arx),jaccard (SGPT+class-zbm)
0,0.122598,0.08261,0.163963,0.144591,0.068801,0.151106,0.175873,0.067823,0.165692,0.169663,0.094112,0.174731,0.169779,0.173467,0.211506,0.104857,0.176334,0.19037,0.066184,0.162718,0.177216,0.174519,0.150766,0.10219,0.167447


In [41]:
model_print_msc_overlap(eval_models, X_test, best_3, level=3)

Bert+MP
	mind 1 match 0.56987203851514
	0% overlap 0.43012796148486
	(0%-50%) 0.28850295783020985
	(0%-100%) 0.06278201717198297
	100% 0.028603728717753803
Bert+MP+class-arx
	mind 1 match 0.43827539494586243
	0% overlap 0.5617246050541376
	(0%-50%) 0.2460602871093179
	(0%-100%) 0.035074895964291634
	100% 0.016255883986784785
Bert+MP+class-zbm
	mind 1 match 0.6599810932764183
	0% overlap 0.34001890672358176
	(0%-50%) 0.3069418861893206
	(0%-100%) 0.08651287898722335
	100% 0.04223801031098637
Mathbert+CLS
	mind 1 match 0.6282294925396408
	0% overlap 0.3717705074603592
	(0%-50%) 0.3010944459063045
	(0%-100%) 0.07864807180656667
	100% 0.03550370825171281
Mathbert+MP+class-arx
	mind 1 match 0.39225603991852565
	0% overlap 0.6077439600814744
	(0%-50%) 0.22796245943338303
	(0%-100%) 0.030026605853287723
	100% 0.012474539270434367
Mathbert+MP+class-zbm
	mind 1 match 0.627245173425333
	0% overlap 0.37275482657466696
	(0%-50%) 0.29656267968696703
	(0%-100%) 0.08058747283376702
	100% 0.0379986161

In [15]:
model_msc_overlap_separate_params(eval_models, X_test, best_3, level=3, prediction_col="MSC")

,MSC overlap (Bert+MP),MSC overlap (Bert+MP+class-arx),MSC overlap (Bert+MP+class-zbm),MSC overlap (Mathbert+CLS),MSC overlap (Mathbert+MP+class-arx),MSC overlap (Mathbert+MP+class-zbm),MSC overlap (Bert+TSDAE+MP),MSC overlap (Bert+TSDAE+MP+class-arx),MSC overlap (Bert+TSDAE+MP+class-zbm),MSC overlap (Bert+MLM+MP),MSC overlap (Bert+MLM+MP+class-arx),MSC overlap (Bert+MLM+MP+class-zbm),MSC overlap (Bert+MLM+MP+class-zbm+anch-arx),MSC overlap (Bert+MLM+MP+class-arx+anch-arx+class-zbm),MSC overlap (SBert),MSC overlap (SBert+ret_class-arx),MSC overlap (SBert+ret_class-zbm),MSC overlap (Specter 2+MP),MSC overlap (Specter 2.0+MP+class-arx),MSC overlap (Specter 2.0+MP+class-zbm),MSC overlap (Specter 2+MP+class-zbm+anch-arx),MSC overlap (Specter 2+MP+anch-arx+class-arx+class-zbm),MSC overlap (SGPT),MSC overlap (SGPT+class-arx),MSC overlap (SGPT+class-zbm)
0,0.249841,0.180338,0.303377,0.283509,0.158258,0.285955,0.329425,0.154049,0.305185,0.320416,0.200714,0.319012,0.312353,0.317714,0.375902,0.214678,0.320261,0.350568,0.15215,0.303227,0.322811,0.319689,0.289723,0.212165,0.310026


## X_eval

### MSC1

In [11]:
model_msc_overlap_jaccard(eval_models, X_eval, best_1, level=1, prediction_col="MSC")

,jaccard (Bert+MP),jaccard (Bert+MP+class-arx),jaccard (Bert+MP+class-zbm),jaccard (Mathbert+CLS),jaccard (Mathbert+MP+class-arx),jaccard (Mathbert+MP+class-zbm),jaccard (Bert+TSDAE+MP),jaccard (Bert+TSDAE+MP+class-arx),jaccard (Bert+TSDAE+MP+class-zbm),jaccard (Bert+MLM+MP),jaccard (Bert+MLM+MP+class-arx),jaccard (Bert+MLM+MP+class-zbm),jaccard (Bert+MLM+MP+class-zbm+anch-arx),jaccard (Bert+MLM+MP+class-arx+anch-arx+class-zbm),jaccard (SBert),jaccard (SBert+ret_class-arx),jaccard (SBert+ret_class-zbm),jaccard (Specter 2+MP),jaccard (Specter 2.0+MP+class-arx),jaccard (Specter 2.0+MP+class-zbm),jaccard (Specter 2+MP+class-zbm+anch-arx),jaccard (Specter 2+MP+anch-arx+class-arx+class-zbm),jaccard (SGPT),jaccard (SGPT+class-arx),jaccard (SGPT+class-zbm)
0,0.393382,0.346839,0.437146,0.441942,0.340467,0.423651,0.478241,0.326933,0.441853,0.488571,0.393207,0.470627,0.463295,0.474125,0.529924,0.385325,0.45914,0.508257,0.333798,0.44935,0.470527,0.471916,0.422866,0.369386,0.448703


In [17]:
model_msc_overlap_separate_params(eval_models, X_eval, best_1, level=1, prediction_col="MSC")

,MSC overlap (Bert+MP),MSC overlap (Bert+MP+class-arx),MSC overlap (Bert+MP+class-zbm),MSC overlap (Mathbert+CLS),MSC overlap (Mathbert+MP+class-arx),MSC overlap (Mathbert+MP+class-zbm),MSC overlap (Bert+TSDAE+MP),MSC overlap (Bert+TSDAE+MP+class-arx),MSC overlap (Bert+TSDAE+MP+class-zbm),MSC overlap (Bert+MLM+MP),MSC overlap (Bert+MLM+MP+class-arx),MSC overlap (Bert+MLM+MP+class-zbm),MSC overlap (Bert+MLM+MP+class-zbm+anch-arx),MSC overlap (Bert+MLM+MP+class-arx+anch-arx+class-zbm),MSC overlap (SBert),MSC overlap (SBert+ret_class-arx),MSC overlap (SBert+ret_class-zbm),MSC overlap (Specter 2+MP),MSC overlap (Specter 2.0+MP+class-arx),MSC overlap (Specter 2.0+MP+class-zbm),MSC overlap (Specter 2+MP+class-zbm+anch-arx),MSC overlap (Specter 2+MP+anch-arx+class-arx+class-zbm),MSC overlap (SGPT),MSC overlap (SGPT+class-arx),MSC overlap (SGPT+class-zbm)
0,0.604399,0.548772,0.644375,0.651692,0.53962,0.62856,0.68963,0.52403,0.647861,0.69691,0.597775,0.674317,0.667408,0.67736,0.730564,0.590379,0.66404,0.71329,0.532644,0.653352,0.673921,0.675937,0.632958,0.573258,0.655511


In [42]:
model_print_msc_overlap(eval_models, X_eval, best_1, level=1)

Bert+MP
	mind 1 match 0.7933612061320157
	0% overlap 0.20663879386798428
	(0%-50%) 0.0646824352639632
	(0%-100%) 0.06015066904462572
	100% 0.41770215088345075
Bert+MP+class-arx
	mind 1 match 0.7409389039947762
	0% overlap 0.2590610960052237
	(0%-50%) 0.0707052987554698
	(0%-100%) 0.05366975606428286
	100% 0.3632624818485708
Bert+MP+class-zbm
	mind 1 match 0.8291377949302693
	0% overlap 0.17086220506973074
	(0%-50%) 0.05777270999619916
	(0%-100%) 0.06770361274352152
	100% 0.45656813729789786
Mathbert+CLS
	mind 1 match 0.8375483632040075
	0% overlap 0.16245163679599256
	(0%-50%) 0.057285423305947825
	(0%-100%) 0.0681714079661628
	100% 0.46234735744427874
Mathbert+MP+class-arx
	mind 1 match 0.729935970528901
	0% overlap 0.27006402947109903
	(0%-50%) 0.06941886189320624
	(0%-100%) 0.05402060248126383
	100% 0.35537818320030407
Mathbert+MP+class-zbm
	mind 1 match 0.8098997163991463
	0% overlap 0.19010028360085374
	(0%-50%) 0.05689559395374675
	(0%-100%) 0.06717734311805007
	100% 0.4439279205

### MSC2

In [12]:
model_msc_overlap_jaccard(eval_models, X_eval, best_2, level=2, prediction_col="MSC")

,jaccard (Bert+MP),jaccard (Bert+MP+class-arx),jaccard (Bert+MP+class-zbm),jaccard (Mathbert+CLS),jaccard (Mathbert+MP+class-arx),jaccard (Mathbert+MP+class-zbm),jaccard (Bert+TSDAE+MP),jaccard (Bert+TSDAE+MP+class-arx),jaccard (Bert+TSDAE+MP+class-zbm),jaccard (Bert+MLM+MP),jaccard (Bert+MLM+MP+class-arx),jaccard (Bert+MLM+MP+class-zbm),jaccard (Bert+MLM+MP+class-zbm+anch-arx),jaccard (Bert+MLM+MP+class-arx+anch-arx+class-zbm),jaccard (SBert),jaccard (SBert+ret_class-arx),jaccard (SBert+ret_class-zbm),jaccard (Specter 2+MP),jaccard (Specter 2.0+MP+class-arx),jaccard (Specter 2.0+MP+class-zbm),jaccard (Specter 2+MP+class-zbm+anch-arx),jaccard (Specter 2+MP+anch-arx+class-arx+class-zbm),jaccard (SGPT),jaccard (SGPT+class-arx),jaccard (SGPT+class-zbm)
0,0.237471,0.190872,0.286,0.274989,0.178004,0.275125,0.30649,0.169679,0.28993,0.311204,0.218739,0.310402,0.303845,0.309119,0.355431,0.220729,0.303437,0.331514,0.172181,0.29382,0.30971,0.309868,0.268142,0.213367,0.293215


In [19]:
model_msc_overlap_separate_params(eval_models, X_eval, best_2, level=2, prediction_col="MSC")

,MSC overlap (Bert+MP),MSC overlap (Bert+MP+class-arx),MSC overlap (Bert+MP+class-zbm),MSC overlap (Mathbert+CLS),MSC overlap (Mathbert+MP+class-arx),MSC overlap (Mathbert+MP+class-zbm),MSC overlap (Bert+TSDAE+MP),MSC overlap (Bert+TSDAE+MP+class-arx),MSC overlap (Bert+TSDAE+MP+class-zbm),MSC overlap (Bert+MLM+MP),MSC overlap (Bert+MLM+MP+class-arx),MSC overlap (Bert+MLM+MP+class-zbm),MSC overlap (Bert+MLM+MP+class-zbm+anch-arx),MSC overlap (Bert+MLM+MP+class-arx+anch-arx+class-zbm),MSC overlap (SBert),MSC overlap (SBert+ret_class-arx),MSC overlap (SBert+ret_class-zbm),MSC overlap (Specter 2+MP),MSC overlap (Specter 2.0+MP+class-arx),MSC overlap (Specter 2.0+MP+class-zbm),MSC overlap (Specter 2+MP+class-zbm+anch-arx),MSC overlap (Specter 2+MP+anch-arx+class-arx+class-zbm),MSC overlap (SGPT),MSC overlap (SGPT+class-arx),MSC overlap (SGPT+class-zbm)
0,0.425908,0.35664,0.480116,0.472059,0.338683,0.466444,0.513116,0.326467,0.485142,0.515817,0.395547,0.508061,0.50094,0.507391,0.562571,0.396851,0.500197,0.53809,0.330645,0.488227,0.50764,0.507517,0.462281,0.388236,0.489631


In [43]:
model_print_msc_overlap(eval_models, X_eval, best_2, level=2)

Bert+MP
	mind 1 match 0.7126178015573683
	0% overlap 0.28738219844263174
	(0%-50%) 0.18207954467931664
	(0%-100%) 0.09798360767573995
	100% 0.17442914364237055
Bert+MP+class-arx
	mind 1 match 0.6237659464569385
	0% overlap 0.3762340535430615
	(0%-50%) 0.18284945764991375
	(0%-100%) 0.07342435848707228
	100% 0.13483222719254645
Bert+MP+class-zbm
	mind 1 match 0.7802434484304496
	0% overlap 0.21975655156955043
	(0%-50%) 0.1774600668557339
	(0%-100%) 0.11995049167227047
	100% 0.20481634164644427
Mathbert+CLS
	mind 1 match 0.770955764114259
	0% overlap 0.22904423588574102
	(0%-50%) 0.17975031429991523
	(0%-100%) 0.11506787903595202
	100% 0.200362541297547
Mathbert+MP+class-arx
	mind 1 match 0.5970041614283348
	0% overlap 0.40299583857166527
	(0%-50%) 0.1787270122503874
	(0%-100%) 0.07098792503581557
	100% 0.1250475104522995
Mathbert+MP+class-zbm
	mind 1 match 0.7580231753549883
	0% overlap 0.24197682464501163
	(0%-50%) 0.17220711633482444
	(0%-100%) 0.11859583467337173
	100% 0.198062548119

### MSC3

In [13]:
model_msc_overlap_jaccard(eval_models, X_eval, best_3, level=3, prediction_col="MSC")

,jaccard (Bert+MP),jaccard (Bert+MP+class-arx),jaccard (Bert+MP+class-zbm),jaccard (Mathbert+CLS),jaccard (Mathbert+MP+class-arx),jaccard (Mathbert+MP+class-zbm),jaccard (Bert+TSDAE+MP),jaccard (Bert+TSDAE+MP+class-arx),jaccard (Bert+TSDAE+MP+class-zbm),jaccard (Bert+MLM+MP),jaccard (Bert+MLM+MP+class-arx),jaccard (Bert+MLM+MP+class-zbm),jaccard (Bert+MLM+MP+class-zbm+anch-arx),jaccard (Bert+MLM+MP+class-arx+anch-arx+class-zbm),jaccard (SBert),jaccard (SBert+ret_class-arx),jaccard (SBert+ret_class-zbm),jaccard (Specter 2+MP),jaccard (Specter 2.0+MP+class-arx),jaccard (Specter 2.0+MP+class-zbm),jaccard (Specter 2+MP+class-zbm+anch-arx),jaccard (Specter 2+MP+anch-arx+class-arx+class-zbm),jaccard (SGPT),jaccard (SGPT+class-arx),jaccard (SGPT+class-zbm)
0,0.121825,0.082288,0.163742,0.14453,0.069269,0.150961,0.174562,0.068042,0.165246,0.168995,0.094271,0.174921,0.169495,0.172997,0.210683,0.104343,0.175296,0.18962,0.065879,0.162611,0.177321,0.173577,0.150504,0.102332,0.167883


In [21]:
model_msc_overlap_separate_params(eval_models, X_eval, best_3, level=3, prediction_col="MSC")

,MSC overlap (Bert+MP),MSC overlap (Bert+MP+class-arx),MSC overlap (Bert+MP+class-zbm),MSC overlap (Mathbert+CLS),MSC overlap (Mathbert+MP+class-arx),MSC overlap (Mathbert+MP+class-zbm),MSC overlap (Bert+TSDAE+MP),MSC overlap (Bert+TSDAE+MP+class-arx),MSC overlap (Bert+TSDAE+MP+class-zbm),MSC overlap (Bert+MLM+MP),MSC overlap (Bert+MLM+MP+class-arx),MSC overlap (Bert+MLM+MP+class-zbm),MSC overlap (Bert+MLM+MP+class-zbm+anch-arx),MSC overlap (Bert+MLM+MP+class-arx+anch-arx+class-zbm),MSC overlap (SBert),MSC overlap (SBert+ret_class-arx),MSC overlap (SBert+ret_class-zbm),MSC overlap (Specter 2+MP),MSC overlap (Specter 2.0+MP+class-arx),MSC overlap (Specter 2.0+MP+class-zbm),MSC overlap (Specter 2+MP+class-zbm+anch-arx),MSC overlap (Specter 2+MP+anch-arx+class-arx+class-zbm),MSC overlap (SGPT),MSC overlap (SGPT+class-arx),MSC overlap (SGPT+class-zbm)
0,0.247706,0.17958,0.302502,0.28261,0.158542,0.28446,0.327319,0.154363,0.304263,0.318684,0.200591,0.318321,0.31122,0.316037,0.374294,0.213158,0.318198,0.349436,0.151278,0.302072,0.322165,0.317243,0.288754,0.211446,0.309684


In [44]:
model_print_msc_overlap(eval_models, X_eval, best_3, level=3)

Bert+MP
	mind 1 match 0.5666559463594811
	0% overlap 0.43334405364051887
	(0%-50%) 0.28981863189388846
	(0%-100%) 0.06159303764776969
	100% 0.02808720482608738
Bert+MP+class-arx
	mind 1 match 0.4367940434074984
	0% overlap 0.5632059565925016
	(0%-50%) 0.24728824956875128
	(0%-100%) 0.03537701371224746
	100% 0.016138935181124463
Bert+MP+class-zbm
	mind 1 match 0.6585484704070793
	0% overlap 0.3414515295929207
	(0%-50%) 0.3058308725355476
	(0%-100%) 0.08854973735247396
	100% 0.04081513317545245
Mathbert+CLS
	mind 1 match 0.6292722860567787
	0% overlap 0.37072771394322135
	(0%-50%) 0.3049732479607052
	(0%-100%) 0.07797561617401982
	100% 0.03457786354023526
Mathbert+MP+class-arx
	mind 1 match 0.3931818846300032
	0% overlap 0.6068181153699967
	(0%-50%) 0.22924889629564657
	(0%-100%) 0.02979270824196708
	100% 0.012045726983013186
Mathbert+MP+class-zbm
	mind 1 match 0.6264070403181008
	0% overlap 0.37359295968189926
	(0%-50%) 0.29794657388728085
	(0%-100%) 0.0820688243721311
	100% 0.036263875